# 基于 LangChain 框架的 RAG 实现

## 数据准备

### 加载原始文档

使用 LangChain 的 TextLoader 加载 Markdown 文件。

In [ ]:
from langchain_community.document_loaders import TextLoader

markdown_path="data/easy-rl-chapter1.md"
loader = TextLoader(markdown_path)
docs = loader.load()


### 文本分块

将长文档被分割成较小的、可管理的文本块（chunks）。

当不指定参数初始化 RecursiveCharacterTextSplitter() 时，其默认行为旨在最大程度保留文本的语义结构。

- 默认分隔符与语义保留: 按顺序尝试使用一系列预设的分隔符 ["\n\n" (段落), "\n" (行), " " (空格), "" (字符)] 来递归分割文本。这种策略的目的是尽可能保持段落、句子和单词的完整性，因为它们通常是语义上最相关的文本单元，直到文本块达到目标大小。

- 保留分隔符: 默认情况下 (keep_separator=True)，分隔符本身会被保留在分割后的文本块中。
  
- 默认块大小与重叠: 使用其基类 TextSplitter 中定义的默认参数 chunk_size=4000（块大小）和 chunk_overlap=200（块重叠）。这些参数确保文本块符合预定的大小限制，并通过重叠来减少上下文信息的丢失。

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
texts = text_splitter.split_documents(docs)


## 索引构建

数据准备完成后，接下来构建向量索引

### 初始化中文嵌入模型

使用 OpenAIEmbeddings 初始化中文嵌入模型。

嵌入模型获取：https://cloud.siliconflow.cn/me/models?types=embedding

In [36]:
from dotenv import load_dotenv
import os
load_dotenv() 

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model=os.getenv("SiliconFlow_EMBEDDING_MODEL"),
    api_key=os.getenv("API_KEY"),
    base_url=os.getenv("SiliconFlow_BASE_URL"),
)


### 构建向量存储

将分割后的文本块 (texts) 通过初始化好的嵌入模型转换为向量表示.

使用 InMemoryVectorStore 将这些向量及其对应的原始文本内容添加进去，从而在内存中构建出一个向量索引。

In [28]:
from langchain_core.vectorstores import InMemoryVectorStore

vectorstore = InMemoryVectorStore(embeddings)
vectorstore.add_documents(texts)


['56ed2256-4faa-4153-a93e-bf58f7d587a3',
 'a591384c-ab6d-4e69-8f92-4f1a6e9a71ab',
 '3d224967-08a9-407e-a700-2fe7ab4484a6',
 '540ecbf9-06ec-4a6d-a1e1-a6c4d3ee1346',
 'f4f4d3fa-a03a-4f3c-a442-3c5d284df1d0',
 '04cafc30-4430-4f48-baf4-56a8e7366aff',
 'e0751ddf-4f93-4ad0-9889-863b7573b2e4']

## 查询与检索

索引构建完毕后，便可以使用 similarity_search 方法针对用户问题进行查询与检索。

In [ ]:
question = "文中举了哪些例子？"

retrieved_docs = vectorstore.similarity_search(question, k=3)


将检索到的多个文本块合并成一个单一的字符串，并使用双换行符 ("\n\n") 分隔各个块，便形成了一个上下文信息字符串，可供大语言模型参考。

In [ ]:
docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)


双换行符通常代表段落的结束和新段落的开始，能够更清晰地在语义上区分这些独立的文本片段。

这种格式有助于LLM将每个块视为一个独立的上下文来源，从而更好地理解和利用这些信息来生成回答。

## 生成集成

将检索到的上下文与用户问题结合，利用大语言模型（LLM）生成答案。

### 配置大语言模型

这里使用 LangChain 调用大语言模型（LLM）。

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv() 

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model=os.getenv("CHAT_MODEL"),
    api_key=os.getenv("API_KEY"),
    base_url=os.getenv("SiliconFlow_BASE_URL"),
)


### 构建提示词模板

使用 ChatPromptTemplate.from_template 创建一个结构化的提示模板。

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""请根据下面提供的上下文信息来回答问题。
请确保你的回答完全基于这些上下文。
如果上下文中没有足够的信息来回答问题，请直接告知：“抱歉，我无法根据提供的上下文找到相关信息来回答此问题。”

上下文:
{context}

问题: {question}

回答:"""
)


### 调用 LLM 生成答案并输出

In [34]:
answer = llm.invoke(prompt.format(question=question, context=docs_content))
print(answer)


content='\n文中举了以下例子：\n\n1. **自然界的例子**：羚羊通过试错学习站立和奔跑。  \n2. **金融领域的例子**：股票交易中通过买卖股票来学习最大化奖励。  \n3. **游戏相关的例子**：  \n   - 打砖块游戏（雅达利游戏中的 Breakout）  \n   - 乒乓球游戏（雅达利游戏中的 Pong）  \n   - 多种经典游戏的动作与观测量解释（如 Atari 游戏的动作空间问题）。  \n4. **Gym 库中的经典环境**：  \n   - 多种控制类游戏的例子（如 Acrobot、CartPole-v0、MountainCar-v0、Taxi-v3）。  \n   - 包括可以观察到观测空间、动作空间、奖励机制的示例，例如 CartPole-v0 中的杆的平衡问题和 MountainCar-v0 中的车辆上山问题。  \n\n其他例子还有迷宫问题（用于解释强化学习策略和价值），以及自然领域中的图像分类案例以对比监督学习与强化学习的区别。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 576, 'prompt_tokens': 6030, 'total_tokens': 6606, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 346, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'deepseek-ai/DeepSeek-R1-0528-Qwen3-8B', 'system_fingerprint': '', 'id': '019a58965a1a229ac2497ac30cac7ced', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--6b3eb2f5-1077-476a-a